<a href="https://colab.research.google.com/github/samitha278/multilayer-perceptron-char-lm/blob/main/mlp_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# download the names.txt file from github
!wget https://github.com/samitha278/multilayer-perceptron-char-lm/blob/main/data/names.txt

--2025-08-23 13:08:19--  https://github.com/samitha278/multilayer-perceptron-char-lm/blob/main/data/names.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘names.txt’

names.txt               [ <=>                ] 560.10K  --.-KB/s    in 0.08s   

2025-08-23 13:08:19 (6.77 MB/s) - ‘names.txt’ saved [573545]



In [3]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

2090
392942
['', '', '', '', '', '', '<!DOCTYPE html>', '<html']


In [4]:
#character mapping
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: ' ', 2: '!', 3: '"', 4: '#', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 0: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '<', 29: '=', 30: '>', 31: '?', 32: '@', 33: 'A', 34: 'B', 35: 'C', 36: 'D', 37: 'E', 38: 'F', 39: 'G', 40: 'H', 41: 'I', 42: 'J', 43: 'K', 44: 'L', 45: 'M', 46: 'N', 47: 'O', 48: 'P', 49: 'Q', 50: 'R', 51: 'S', 52: 'T', 53: 'U', 54: 'V', 55: 'W', 56: 'X', 57: 'Y', 58: 'Z', 59: '[', 60: ']', 61: '_', 62: '`', 63: 'a', 64: 'b', 65: 'c', 66: 'd', 67: 'e', 68: 'f', 69: 'g', 70: 'h', 71: 'i', 72: 'j', 73: 'k', 74: 'l', 75: 'm', 76: 'n', 77: 'o', 78: 'p', 79: 'q', 80: 'r', 81: 's', 82: 't', 83: 'u', 84: 'v', 85: 'w', 86: 'x', 87: 'y', 88: 'z', 89: '{', 90: '}', 91: '·', 92: '’', 93: '…'}
93


### Build the dataset

In [5]:
block_size = 3 # context length

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y


random.seed(278)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([146293, 3]) torch.Size([146293])
torch.Size([19691, 3]) torch.Size([19691])
torch.Size([407551, 3]) torch.Size([407551])


### Build MLP

In [6]:
n_embd = 10      # dimensionality of the character embedding vectors
n_hidden = 64    # number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)
C  = torch.randn((vocab_size, n_embd),            generator=g)


# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 useless because of BN


# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1


# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1



parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
  p.requires_grad = True

9087


## Forward pass

In [7]:
batch_size = 32
n = batch_size
#minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [8]:

emb = C[Xb]                         # characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors


# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation

# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # Bessel's correction
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv

hpreact = bngain * bnraw + bnbias


# # hidden layer - Non-linearity
h = torch.tanh(hpreact)


# Linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability

counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1

probs = counts * counts_sum_inv
logprobs = probs.log()

loss = -logprobs[range(n), Yb].mean()



In [9]:
# PyTorch backward pass
for p in parameters:
  p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(4.5734, grad_fn=<NegBackward0>)

### Comparing manual gradients to PyTorch gradients

In [10]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

## Backprop - manually



In [11]:
logprobs.shape

torch.Size([32, 93])

In [12]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n),Yb] = -1.0/n



dprobs = dlogprobs * (1.0/probs)

dcounts = dprobs * counts_sum_inv

dcounts_sum_inv = (dprobs * counts).sum(1,keepdims = True)


dcounts_sum = dcounts_sum_inv * (-counts_sum**-2)

dcounts += dcounts_sum * torch.ones_like(counts)


dnorm_logits = dcounts * counts  #norm_logits.exp()



dlogit_maxes = dnorm_logits * -1 * torch.ones_like(logits)




# dlogits = dnorm_logits
# dh = dlogits @ W2.T
# dW2 = dlogits.T  @ h
# db2 = dlogits





cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)



logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: False | approximate: False | maxdiff: 0.031169598922133446


In [13]:
logit_maxes

tensor([[1.2956],
        [1.0431],
        [1.0291],
        [0.9757],
        [1.2925],
        [1.3569],
        [1.2586],
        [1.5214],
        [1.6282],
        [1.2867],
        [1.2438],
        [1.3182],
        [1.1812],
        [1.5553],
        [0.9487],
        [1.3382],
        [1.0264],
        [1.1072],
        [1.5294],
        [1.1754],
        [1.4103],
        [1.5553],
        [1.1048],
        [1.6779],
        [1.6396],
        [1.1186],
        [1.1915],
        [1.2986],
        [1.5553],
        [0.8035],
        [0.8696],
        [1.3355]], grad_fn=<MaxBackward0>)

In [14]:
dprobs * counts

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.3944,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MulBackward0>)

In [15]:
dlogprobs

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0312,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [16]:
dprobs

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-2.5177,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MulBackward0>)

In [17]:
dcounts_sum_inv

tensor([[-0.9447],
        [-1.0509],
        [-1.0467],
        [-1.1663],
        [-0.9413],
        [-0.8533],
        [-0.9385],
        [-0.6914],
        [-0.6044],
        [-0.8829],
        [-0.9372],
        [-0.9675],
        [-1.0209],
        [-0.7557],
        [-1.2060],
        [-0.8738],
        [-1.1517],
        [-1.0705],
        [-0.7144],
        [-0.9766],
        [-0.8791],
        [-0.7557],
        [-1.0333],
        [-0.6549],
        [-0.6978],
        [-1.0695],
        [-0.9309],
        [-0.8978],
        [-0.7557],
        [-1.3944],
        [-1.2086],
        [-0.8566]], grad_fn=<SumBackward1>)

In [18]:
dprobs * counts

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.3944,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MulBackward0>)

In [19]:
dlogprobs * (1.0/probs) * counts

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.3944,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MulBackward0>)

In [20]:
counts_sum_inv

tensor([[0.0331],
        [0.0297],
        [0.0299],
        [0.0268],
        [0.0332],
        [0.0366],
        [0.0333],
        [0.0452],
        [0.0517],
        [0.0354],
        [0.0333],
        [0.0323],
        [0.0306],
        [0.0414],
        [0.0259],
        [0.0358],
        [0.0271],
        [0.0292],
        [0.0437],
        [0.0320],
        [0.0355],
        [0.0414],
        [0.0302],
        [0.0477],
        [0.0448],
        [0.0292],
        [0.0336],
        [0.0348],
        [0.0414],
        [0.0224],
        [0.0259],
        [0.0365]], grad_fn=<PowBackward0>)

In [21]:
dlogprobs * (1.0/probs)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-2.5177,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<MulBackward0>)

In [22]:
probs

tensor([[0.0130, 0.0097, 0.0287,  ..., 0.0099, 0.0062, 0.0038],
        [0.0065, 0.0179, 0.0114,  ..., 0.0029, 0.0063, 0.0077],
        [0.0054, 0.0149, 0.0043,  ..., 0.0094, 0.0067, 0.0109],
        ...,
        [0.0124, 0.0067, 0.0161,  ..., 0.0120, 0.0109, 0.0105],
        [0.0060, 0.0093, 0.0106,  ..., 0.0079, 0.0122, 0.0041],
        [0.0091, 0.0064, 0.0083,  ..., 0.0145, 0.0128, 0.0115]],
       grad_fn=<MulBackward0>)

In [23]:
1.0/probs

tensor([[ 76.7810, 103.1417,  34.8359,  ..., 101.1068, 161.6606, 264.5447],
        [153.5139,  56.0160,  87.6383,  ..., 344.0364, 158.0789, 130.6225],
        [184.1756,  66.9186, 235.2254,  ..., 106.6008, 148.8115,  91.7553],
        ...,
        [ 80.5650, 148.2551,  62.0806,  ...,  83.4420,  91.8387,  95.6044],
        [166.4879, 106.9857,  94.2314,  ..., 126.8341,  82.0014, 241.9802],
        [109.9942, 155.3180, 120.5484,  ...,  69.1642,  78.2783,  86.6866]],
       grad_fn=<MulBackward0>)